### Import the modules

In [6]:
import os
from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display
from openai import OpenAI

### Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### A quick call to a Frontier model to get started.

In [3]:
message = "Hello, GPT! This is my first ever message to you. Hi!"

messages = [
    {'role': 'user', 'content': message}
]

messages

[{'role': 'user',
  'content': 'Hello, GPT! This is my first ever message to you. Hi!'}]

In [4]:
openai_client = OpenAI()

response = openai_client.chat.completions.create(
    model="gpt-5-nano",
    messages= messages
)

response.choices[0].message.content

'Hi! Nice to meet you. Welcome to the chat.\n\nI can help with lots of things—explanations, writing, planning, coding, brainstorming, learning new topics, and more. What would you like to do today?\n\nA few quick ideas to get started:\n- Explain something in simple terms (e.g., photosynthesis)\n- Help draft or edit a message, email, or essay\n- Brainstorm ideas for a project, story, or trip\n- Solve a math problem or debug code\n- Summarize or translate text\n- Create a study plan or learn a new topic step by step\n\nTell me your interests or a specific task, and I’ll tailor the response.'

### Our first project

In [7]:
ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
AI Curriculum
Proficient AI Engineer
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
I will happily drone on for hours about LLMs to anyone in my vicinity. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To my total joy (and shock) they’ve become best-selling, top-rated courses, with 400,000 enrolled across 190

### Types of prompts

1. System prompt that tells them what task they are performing and what tone they should use.

2. User prompt -- the conversation starter that they should reply to.

In [8]:
# Define our system prompt.

system_prompt = """
You are a snarky assistant that analyzes the contents 
of a website, and provides a short, snarky, humorous 
summary, ignoring text that might be navigation
related. Respond in markdown. Do not wrap the markdown
in a code block - respond just with the markdown.
"""

In [9]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize 
these too.
"""

### Messages

The API from OpenAI expects to receive messages in a particular structure. 

In [12]:
messages = [
    {'role': 'system', 'content': "You are a snarky assistant"},
    {'role': 'user', 'content': "what is 2 + 2"}
]

response = openai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages = messages
)

response.choices[0].message.content

'Oh, really? You need me to do basic math? Fine, 2 + 2 equals... drumroll, please... 4!'

### Create a messages function

In [13]:
def message_for(website):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_prefix + website}
    ]


In [14]:
message_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents \nof a website, and provides a short, snarky, humorous \nsummary, ignoring text that might be navigation\nrelated. Respond in markdown. Do not wrap the markdown\nin a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize \nthese too.\nHome - Edward Donner\n\nHome\nAI Curriculum\nProficient AI Engineer\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.i

### Calling OpenAI using API.

In [17]:
def summarizer(url):
    website = fetch_website_contents(url)

    response = openai_client.chat.completions.create(
        model= "gpt-4.1-nano",
        messages= message_for(website)
    )

    return response.choices[0].message.content

In [18]:
summarizer("https://edwarddonner.com")

'Welcome to Edward Donner’s digital playground where AI isn’t just a tool, but a hobby, a passion, and apparently an obsession. From launching handy Udemy courses with a whopping 400,000 students to co-founding Nebula.io — because plain old AI isn’t enough — Ed’s here to make sure you’re as fascinated with large language models as he is. Read his latest updates: Vibe Coder to Agentic Engineer, AI voice agents, and the energy of AI live events. Plus, he seems to think email outreach is a gentle way to stay in touch — lucky you!'

In [22]:
def display_summary(url):
    summary = summarizer(url)
    display(Markdown(summary))

In [23]:
display_summary("https://edwarddonner.com")

Welcome to Edward Donner’s digital playground where coding, AI, and amateur electronic music collide—because who doesn't want that? Ed's a founder, CTO, and a self-professed LLM enthusiast with best-selling Udemy courses that have global reach (hello, 190 countries). He's also into battles of wits between language models in a curious arena called Outsmart—think of it as "AI cage fight" for the nerdy at heart. Recent updates include slick titles like "Vibe Coder to Agentic Engineer" and "Create Agents and Voice Agents," so stay tuned for more AI adventures. Basically, if you're into AI, giggles, or just want to say you know the guy who makes LLMs fight each other—this site is your hub.

In [24]:
display_summary("https://cnn.com")

Wow, look at CNN, the ultimate click-baity playground with endless categories because they really want to distract you from the fact that you're just trying to find the news. From politics and world affairs to fashion and fitness, apparently they have it all — minus the substance. Their feedback form about ads is more engaging than most articles, offering an Olympic-level variety of issues. In their world, "Breaking News" might be a slow-loading video ad, but fear not, you can always switch to the Winter Olympics 2026 or the Ukraine-Russia conflict—because nothing says clarity like a menu of chaos.

In [25]:
display_summary("https://anthropic.com")

Welcome to Anthropic, where the focus is on making AI safe enough to potentially bring about a better world—or at least a less disastrous one. They’re proudly marching toward AI futures that put safety front and center, because apparently technology isn’t scary enough yet. 

Recent highlights include a robotic journey to Mars—because clearly, the future is a giant leap for robot-kind—and a shiny new AI model, Claude Opus 4.6, which promises to be the Swiss Army knife for coding and professional tasks. 

And if you ever wondered what sort of AI twist might be needed for planetary exploration or professional efficiency, they’ve got you covered. Plus, they’re all about transparency with no pesky ads to distract you. Just top-tier AI safety and some space-themed ambitions.